In [1]:
import numpy as np
import os

In [2]:
labels = np.loadtxt('labels.csv')

OSError: labels.csv not found.

In [3]:
rootdir = 'images'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print(file)

0000.png
0001.png
0002.png
0003.png
0004.png
0005.png
0006.png
0007.png
0008.png
0009.png
0010.png
0011.png
0012.png
0013.png
0014.png
0015.png
0016.png
0017.png
0018.png
0020.png
0021.png
0022.png
0023.png
0024.png
0025.png
0026.png
0027.png
0028.png
0029.png
0030.png
0031.png
0032.png
0033.png
0034.png
0035.png
0036.png
0037.png
0038.png
0040.png
0041.png
0042.png
0043.png
0044.png
0045.png
0046.png
0047.png
0048.png
0049.png
0050.png
0051.png
0052.png
0053.png
0054.png
0055.png
0056.png
0057.png
0058.png
0060.png
0061.png
0062.png
0063.png
0064.png
0065.png
0066.png
0067.png
0068.png
0069.png
0070.png
0071.png
0072.png
0073.png
0074.png
0075.png
0076.png
0077.png
0078.png
0080.png
0081.png
0082.png
0083.png
0084.png
0085.png
0086.png
0087.png
0088.png
0089.png
0090.png
0091.png
0092.png
0093.png
0094.png
0095.png
0096.png
0097.png
0098.png
0019.png
0039.png
0059.png
0079.png
0099.png
0119.png
0139.png
0159.png
0179.png
0199.png
0219.png
0239.png
0259.png
0279.png
0299.png
0319.png
0

In [24]:
onlyfiles = [f for f in os.listdir(rootdir) if os.path.isfile(os.path.join(rootdir, f))]

from scipy import ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

train_files = []
y_train = labels

i=0

for _file in onlyfiles:
    train_files.append(_file)

NameError: name 'labels' is not defined

In [20]:
from PIL import Image

dataset = np.ndarray(shape=(len(train_files), 144, 216, 3), dtype=np.float32)

i = 0
for _file in train_files:
    img = load_img(rootdir + "/" + _file)
    img.thumbnail((216, 144))
    x = img_to_array(img)
    x = x.reshape((144, 216, 3))
    x = (x - 128.0) / 128.0
    dataset[i] = x
    i += 1

In [23]:
from sklearn.model_selection import train_test_split

#Splitting 
X_train, X_test, y_train, y_test = train_test_split(dataset, y_train, test_size=0.1, random_state=33)

NameError: name 'y_train' is not defined

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.python.keras.layers.normalization import BatchNormalization

np.random.seed(1000)
#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(144,216,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(17))
model.add(Activation('softmax'))

model.summary()

# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer='adam', metrics=["accuracy"])

W0106 14:02:37.958695 139720573544256 deprecation.py:506] From /home/isuru/.local/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


ValueError: Negative dimension size caused by subtracting 3 from 1 for 'conv2d_3/Conv2D' (op: 'Conv2D') with input shapes: [?,1,6,384], [3,3,384,384].

In [26]:
history = model.fit(X_train, y_train,
          batch_size=2,
          epochs=8,
          validation_data=(X_test, y_test))

Train on 461 samples, validate on 52 samples
Epoch 1/8
461/461 [==============================] - 191s 415ms/sample - loss: 1.9652 - acc: 0.4187 - val_loss: 1.4107 - val_acc: 0.4231
Epoch 2/8
461/461 [==============================] - 186s 404ms/sample - loss: 1.3551 - acc: 0.4295 - val_loss: 1.2443 - val_acc: 0.4231
Epoch 3/8
461/461 [==============================] - 187s 406ms/sample - loss: 1.3407 - acc: 0.4490 - val_loss: 1.2621 - val_acc: 0.4231
Epoch 4/8
461/461 [==============================] - 186s 403ms/sample - loss: 1.3421 - acc: 0.4187 - val_loss: 1.2799 - val_acc: 0.4231
Epoch 5/8
461/461 [==============================] - 191s 414ms/sample - loss: 1.3346 - acc: 0.4403 - val_loss: 1.2221 - val_acc: 0.4231
Epoch 6/8
461/461 [==============================] - 194s 421ms/sample - loss: 1.3297 - acc: 0.4360 - val_loss: 1.2147 - val_acc: 0.4231
Epoch 7/8
461/461 [==============================] - 189s 409ms/sample - loss: 1.3208 - acc: 0.4512 - val_loss: 1.2127 - val_acc: 0.4